# read state_dict

In [2]:
import torch

model_folder = './models/'
state_l = []
for i in range(300, 9901, 300):
    model_name = model_folder + 'nrange_30_50_episode_'+ str(i) + '.pt'
    state_l.append(torch.load(model_name, map_location=torch.device('cpu')))

In [3]:
len(state_l)

33

# load test set

In [4]:
import networkx as nx

graph_folder = './data/'
graph_names = [
    'Crime.txt',
    'Digg.txt',
    'Enron.txt',
    'Epinions.txt',
    'HI-II-14.txt'
]

graph_l = []
for name in graph_names:
    graph_l.append(nx.read_edgelist(graph_folder + name))


In [5]:
for g in graph_l:
    print(len(g.nodes))

829
29652
33696
75877
4165


# evaluate

In [6]:
from Q_network import QNetwork

GAMMA = 1  # decay rate of past observations
UPDATE_TIME = 1000
EMBEDDING_SIZE = 64
# MAX_ITERATION = 500000
LEARNING_RATE = 0.0001  # dai
MEMORY_SIZE = 500000
Alpha = 0.001  # weight of reconstruction loss

# hyperparameters for priority
# epsilon = 0.0000001  # small amount to avoid zero priority
# alpha = 0.6  # [0~1] convert the importance of TD error to priority
# beta = 0.4  # importance-sampling, from initial value increasing to 1
# beta_increment_per_sampling = 0.001
# TD_err_upper = 1.  # clipped abs error

# other network para
N_STEP = 5
NUM_MIN = 30
NUM_MAX = 50
# REG_HIDDEN = 32
BATCH_SIZE = 64
# initialization_stddev = 0.01  # 权重初始化的方差
n_valid = 200
# aux_dim = 4
num_env = 1
# inf = 2147483647 / 2

# embedding method
K = 3
# aggregatorID = 0  # 0:sum; 1:mean; 2:GCN
# embeddingMethod = 1  # 0:structure2vec; 1:graphsage

# my parameter
INPUT_SIZE = 2  # size of input feature, c in Algorithm S2
N_train = 1000
N_episode = 10000
T = 10
C = UPDATE_TIME
UPDATE_GRAPHS = 100
tensor_type = torch.float32
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
from train_utils import prepare_batch
from math import floor
from graph_utils import accumulated_normalized_connectivity
def test(model, graph):
    g_copy:nx.Graph = graph.copy()
    num_remove_nodes = floor(0.01 * len(graph.nodes))
    
    deleted_nodes = []
    while nx.number_of_edges(g_copy) > 0:
        out = model(*prepare_batch(q_for_all=True,
                                    tensor_type=tensor_type,
                                    device=device,
                                    s_t=[torch.tensor(nx.adjacency_matrix(g_copy).todense(), device=device, dtype=tensor_type)]))
        node_sort = list(torch.argsort(torch.squeeze(out, dim=1)))
        node_id = node_sort[:num_remove_nodes]
        node_name = []
        for ni in node_id:
            node_name.append(list(g_copy.nodes)[ni])
        for nn in node_name:
            deleted_nodes.append(nn)
            g_copy.remove_node(nn)
    
    sol = deleted_nodes + list(set(graph.nodes)^set(deleted_nodes))
    return accumulated_normalized_connectivity(sol, graph)

q = QNetwork(INPUT_SIZE, EMBEDDING_SIZE, K, tensor_type, device)

In [8]:
from tqdm import tqdm
result = []
for i in range(len(state_l)):
    result.append([])
for i in tqdm(range(len(state_l))):
    q.load_state_dict(state_l[i])
    for j in range(1):
        result[i].append(test(q, graph_l[j]))

100%|██████████| 33/33 [00:35<00:00,  1.08s/it]


In [10]:
import numpy as np

np.array(result)

array([[0.33120973],
       [0.33204198],
       [0.33166373],
       [0.33237493],
       [0.33243603],
       [0.3323136 ],
       [0.33233662],
       [0.33229022],
       [0.33229275],
       [0.33230754],
       [0.33237102],
       [0.33236533],
       [0.33120568],
       [0.33210123],
       [0.33171588],
       [0.33212034],
       [0.33192174],
       [0.33191142],
       [0.33192632],
       [0.33204528],
       [0.33219768],
       [0.33225382],
       [0.3320153 ],
       [0.33210947],
       [0.33211651],
       [0.33182098],
       [0.33188069],
       [0.33178112],
       [0.33205813],
       [0.33236529],
       [0.33243594],
       [0.3322575 ],
       [0.33243872]])